# 구조화 문장에서 사고원인 추출

### colab 세팅

In [1]:
from google.colab import drive
drive.mount('/content/drive')

parent_path = '/content/drive/Othercomputers/My MacBook Pro/colab/'

import sys
sys.path.append(parent_path + 'preprocess')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!python '/content/drive/Othercomputers/My MacBook Pro/colab/settings.py'
%env OLLAMA_MAX_LOADED_MODELS=2
%env OLLAMA_NUM_PARALLEL=2
!nohup ollama serve &

### 맞춤법 수정 결과 검증 prompt 준비

In [1]:
import pandas as pd
from utils import load_ollama, run_ollama, parallel_process_n
import re
import json
from tqdm import tqdm
tqdm.pandas()

test_df = pd.read_csv('../data/test_06.csv')

### json 형식 검증

In [2]:
llm_response = """
```json
{
 "user_question": {
   "question": "test",
   "answer": "test"
 }
}
```
"""

def is_valid_json_reason(text):
    """
    주어진 텍스트 내의 ```json 코드 블록 내부 내용이 유효한 JSON 형식이며,
    그 JSON 데이터에 '사고 원인' 키가 존재하면 True, 그렇지 않으면 False를 반환합니다.
    """
    # 정규표현식을 사용하여 ```json과 ``` 사이의 내용 추출
    if type(text) != str:
        return False
    match = re.search(r"```json(.*?)```", text, re.DOTALL)
    if match:
        json_str = match.group(1).strip()  # 양쪽 공백 제거
        try:
            data = json.loads(json_str)
            # JSON 데이터가 딕셔너리이고 '사고 원인' 키가 존재하는지 확인
            if isinstance(data, dict) and all('user_question' in data and key in data['user_question'] and isinstance(data['user_question'][key], str) for key in ['question', 'answer']):
                return True
            else:
                return False
        except json.JSONDecodeError:
            return False
    return False

print(is_valid_json_reason(llm_response))

True


In [3]:
is_valid_json_reason(test_df.loc[0]['llm_answer_1'])

False

In [4]:
llm_answer_cols = [col for col in test_df.columns if 'llm_answer' in col]
llm_answer_isvalid_cols = ['isvalid' + col for col in test_df.columns if 'llm_answer' in col]

In [5]:
for answer_col, valid_col in zip(llm_answer_cols, llm_answer_isvalid_cols):
    test_df[valid_col] = test_df[answer_col].apply(lambda x: is_valid_json_reason(x))

In [6]:
test_df[llm_answer_isvalid_cols]

,isvalidllm_answer_1,isvalidllm_answer_2,isvalidllm_answer_3
0,False,False,False
1,True,True,True
2,True,True,True
3,True,True,True
4,True,True,True
...,...,...,...
959,True,True,True
960,True,True,True
961,True,True,True
962,True,True,True


### 적절한 문장 형식 나올 때까지 모델 응답 요청

In [7]:
chain = load_ollama(model="gemma3:4bcp", temperature=1.0, num_predict=1024)

def find_proper_form(row):
    answer = ''
    print(row['id'])
    while not is_valid_json_reason(answer):
        answer = run_ollama(row['rag_summary_prompt'], chain)
        print(answer)
    return answer

for answer_col, valid_col in zip(llm_answer_cols, llm_answer_isvalid_cols):
    test_df.loc[~test_df[valid_col], answer_col] = test_df[~test_df[valid_col]].apply(lambda x: find_proper_form(x), axis=1)

0
```json
{
 "user_question": {
   "question": "발생 배경: 타설 작업 중 콘크리트 펌프 사용, 사고 종류: 콘크리트 펌프 관련 부딪힘 및 전도 사고, 사고 원인: 지반 침하 (아웃트리거 우측 상부 1개소), 아웃트리거 펼친 길이 상이, 타설 위치 건물 끝부분 모서리, 붐대호스 최대 펼침으로 인한 장비 무게 중심 불균형",
   "answer": "재발 방지 대책으로 작업 전 TBM 시행 시 유사 및 동일 공종 작업자 안전교육과 콘크리트 펌프카 점검 항목의 필수 점검을 포함한 수시 위험성 평가를 실시하고, 재해 재발 방지를 위한 대책 수립 및 교육을 철저히 시행하며, 한국전력공사 내 사고 사례 전파와 안전보건협의체, 안전시공보고회의, 합동안전점검 등을 통한 관리감독자 교육을 진행하는 계획. 특히, 지반 침하로 인한 장비 무게 중심 불균형 발생 위험성을 고려하여, 타설 작업 전 지반 상태 점검 및 아웃트리거 길이 제한 등의 사전 안전 조치를 강화하고, 작업자 교육 시 장비 사용 시 주의사항 및 안전 수칙을 명확히 교육하며, 펌프카 사용 시 무게 중심 균형을 고려한 작업 방법을 숙지하도록 하는 계획."
 }
}
```
8


KeyboardInterrupt: 

### json에서 answer 추출

In [ ]:
def extract_json(text):
    # 정규표현식을 사용하여 ```json과 ``` 사이의 내용을 추출
    match = re.search(r"```json(.*?)```", text, re.DOTALL)
    json_str = match.group(1).strip()  # 양쪽 공백 제거
    data = json.loads(json_str)         # JSON 문자열을 딕셔너리로 변환
    return data['user_question']['answer']         # '사고 원인' 키의 값을 반환

for answer_col in llm_answer_cols:
    test_df[answer_col] = test_df[answer_col].progress_apply(extract_json)

  0%|          | 1/964 [00:00<00:00, 3862.16it/s]

None


AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
test_df.to_csv('../data/test_07.csv', index=False)